My Goal
1) Tim's data upload. (sampledata shape)
2) Process to trial based data
3) Get left choice, reward differences, and length differences
4) scale them
5) run the fucking code

In [1]:
import sys
path1 = r'C:\Users\yangsunhwang\Documents\GitHub\DA_maze\DA\Modules'
path2 = r'C:\Users\yangsunhwang\Documents\GitHub\DA_maze\Behavior\Modules'
sys.path += [path1,path2]

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
plt.style.use("seaborn-v0_8-poster")
from multi_rat_da import *
from hexLevelAnalyses import get_sigRats_fromMeanList
from photometryQuantifications import *
from scipy.stats import wilcoxon
%matplotlib qt

# import photometry and behavioral data for alignment and comparison

In [3]:
#saline
seshDict = {"IM-1594":["07252023","07262023","07272023","07282023"],\
            "IM-1372":["06062021","06082021","06092021","06102021"]}
photrats = PhotRats(seshDict)
photrats.directory_prefix="W:/Photometry/"
photrats.load_dfs()
print(photrats)

Memory usage of dataframe is 307.88 MB
Memory usage after optimization is: 73.89 MB
Decreased by 76.0%
Memory usage of dataframe is 322.51 MB
Memory usage after optimization is: 77.40 MB
Decreased by 76.0%
Memory usage of dataframe is 342.88 MB
Memory usage after optimization is: 82.29 MB
Decreased by 76.0%
Memory usage of dataframe is 344.31 MB
Memory usage after optimization is: 82.64 MB
Decreased by 76.0%
Memory usage of dataframe is 289.53 MB
Memory usage after optimization is: 96.99 MB
Decreased by 66.5%
Memory usage of dataframe is 313.86 MB
Memory usage after optimization is: 105.14 MB
Decreased by 66.5%
Memory usage of dataframe is 275.07 MB
Memory usage after optimization is: 92.15 MB
Decreased by 66.5%
Memory usage of dataframe is 210.87 MB
Memory usage after optimization is: 70.64 MB
Decreased by 66.5%


In [6]:
photrats.df

,x,y,port,rwd,block,ref,vel,acc,tri,beamA,...,date,lenAC,lenBC,lenAB,dtop,nom_rwd_a,nom_rwd_b,nom_rwd_c,hexlabels,session
0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,0.0,...,6102021,19.0,17.0,15.0,15.0,90.0,10.0,50.0,-1,8
1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,0.0,...,6102021,19.0,17.0,15.0,15.0,90.0,10.0,50.0,-1,8
2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,0.0,...,6102021,19.0,17.0,15.0,15.0,90.0,10.0,50.0,-1,8
3,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,0.0,...,6102021,19.0,17.0,15.0,15.0,90.0,10.0,50.0,-1,8
4,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,0.0,...,6102021,19.0,17.0,15.0,15.0,90.0,10.0,50.0,-1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105542,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,64.0,0.0,...,6102021,NaN,NaN,NaN,NaN,50.0,90.0,10.0,1,8
1105543,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,64.0,0.0,...,6102021,NaN,NaN,NaN,NaN,50.0,90.0,10.0,1,8
1105544,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,64.0,0.0,...,6102021,NaN,NaN,NaN,NaN,50.0,90.0,10.0,1,8
1105545,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,64.0,0.0,...,6102021,NaN,NaN,NaN,NaN,50.0,90.0,10.0,1,8


In [ ]:
ratSeshDict = {"IM-1594":["07252023","07262023","07272023","07282023"],\
               "IM-1372":["06062021","06082021","06092021","06102021"]}
#{"IM-1478":["07192022","07202022","07242022","07252022","07262022","07272022"]}
hipRats = DecodePhotRats(seshdict=ratSeshDict,hpd_thresh=800,distThresh=15,
        velThresh=10,lowVelThresh=4,xRange=[220,400],yRange=[100,300],thetaBinNum=14)
hipRats.directory_prefix = 'W:/Ephys/'
hipRats.phot_directory_prefix = 'W:/Photometry/'

df = pd.read_csv(loadpath+datName+".csv",index_col=[0])
photrats = PhotRats(None)
photrats.df = reduce_mem_usage(df)
del df
photrats.directory_prefix = loadpath

In [103]:
filepath = "W:/Photometry/"+ rat + '/'#"/Volumes/Tim-1/Photometry Data/"+ rat + '/'
filecount = '0'# input('file suffix number: ')
photLoadpath = filepath+date+'/'

ephysFs = 30000
lfpFs = 1500
photFs = 250
ephysLoadpath = ephys_loadpath#"/Volumes/Tim/Ephys/"+ rat + '/' + date + '/'

#load sampleframe
df = pd.read_csv(photLoadpath+rat+'_'+date+"_h_sampleframe.csv")#)"sampleframe.csv")
#df = (pd.read_pickle(loadpath+"fullDf_fixed1478pos.pkl")[lambda x: (x['rat'] == rat) & (x['date'] == int(date))])
df = reduce_mem_usage(df)
#df.loc[:,"x"] = df.loc[:,"x"].interpolate()
#df.loc[:,"y"] = df.loc[:,"y"].interpolate()

#get barrier IDs
if df.session_type.values[0]=='prob':
    tmat = np.load(photLoadpath+"tmat.npy")
    barriers = np.where(np.mean(tmat,axis=1)==0)[0][1:]
else:
    barriers = []
    for b in range(len(df.block.unique())):
        tmat = np.load(photLoadpath+'tmat_block_'+str(b+1)+'.0.npy')
        barriers.append(np.where(np.mean(tmat,axis=1)==0)[0][1:])

#df.drop(['level_0'],inplace=True,axis=1)
df.reset_index(inplace=True)
#df.drop(['level_0'],inplace=True,axis=1)

if date[-2:]=='23':
    df.loc[:,"vel"] = df.loc[:,"vel"]/(3.14*pix2cm)

if len(df.loc[smpls2TrimFromStart:,:])-len(dToRat) == 0:
    df = df.loc[smpls2TrimFromStart:,:].copy()

df.reset_index(inplace=True)
#df.drop(['level_0'],inplace=True,axis=1)

df.loc[:,"rwd"] = df.loc[:,"rwd"].fillna(0)
df.loc[:,"rwd"] = df.loc[:,"rwd"].astype("uint8")

#add trial number to df
df.loc[:,"port"] = df.port.fillna(-100)
df.loc[:,"port"] = df.loc[:,"port"].astype("int16")

df.loc[:,"tot_tri"] = -1
t = 0
for i in df.loc[df.port!=-100].index:
    df.loc[i,"tot_tri"] = t
    t +=1
df.loc[:,"tot_tri"] = df.loc[:,"tot_tri"].replace(-1,method = 'bfill')

#interpolate missing position data
df.loc[:,"x"] = df.loc[:,"x"].interpolate()
df.loc[:,"y"] = df.loc[:,"y"].interpolate()
df.loc[:,"vel"].interpolate(inplace=True)

Memory usage of dataframe is 387.17 MB
Memory usage after optimization is: 112.92 MB
Decreased by 70.8%


C:\Users\yangsunhwang\AppData\Local\Temp\ipykernel_15644\2446922124.py:54: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df.loc[:,"tot_tri"] = df.loc[:,"tot_tri"].replace(-1,method = 'bfill')


In [ ]:
hipRats.load_dfs()
hipRats.add_totTri2fullDf()

hipRats.df.drop(['Unnamed: 0',"Unnamed: 0.1","470",'nextprob', 'nextp', 'simple_rr',
       'pchosen','level_0', 'green', 'pA', 'pB',
       'pC', 'frame', 'ref', 'fromP'],axis=1,inplace=True)

In [6]:
seshDict = {"IM-1532":["12142022"],"IM-1533":["12142022"]}#{"IM-1434":["04132022","04192022"]}
salRats = PhotRats(seshDict)
salRats.directory_prefix="Z:/Photometry/"
salRats.load_dfs()

Memory usage of dataframe is 351.63 MB
Memory usage after optimization is: 86.15 MB
Decreased by 75.5%
Memory usage of dataframe is 357.70 MB
Memory usage after optimization is: 87.64 MB
Decreased by 75.5%


### load trial-level dataframe

In [3]:
loadpath = r"C:\Users\yangsunhwang\Documents\GitHub\DA_maze\Data" #location of dataset
df = pd.read_csv(loadpath+"\\trialLevelDf.csv")
photrats = PhotRats(None)
photrats.triframe = df.copy()
photrats.triframe.reset_index(inplace=True)
del df
photrats.directory_prefix = loadpath

## create new dataframes with relevant columns for regression, and plot results

In [4]:
get_portHist(photrats)

get_back2samePortTrials(photrats)

add_rewardLagColumns2triframe(photrats)

add_rewardLagColumns2triframeByPort(photrats,lags=5)

add_samePathLagColumns2triframeByPort(photrats,lags=1)

add_leftLastColumn(photrats)

C:\Users\yangsunhwang\Documents\GitHub\DA_maze\DA\Modules\choiceAnalysisFuncs.py:64: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  photrats.triframe.loc[photrats.triframe.session==sesh,["rt-"+str(l)+"_"+s for s in portStrings]].fillna(method='ffill')
C:\Users\yangsunhwang\Documents\GitHub\DA_maze\DA\Modules\choiceAnalysisFuncs.py:64: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  photrats.triframe.loc[photrats.triframe.session==sesh,["rt-"+str(l)+"_"+s for s in portStrings]].fillna(method='ffill')
C:\Users\yangsunhwang\Documents\GitHub\DA_maze\DA\Modules\choiceAnalysisFuncs.py:64: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  photrats.triframe.loc[photrats.triframe.session==sesh,["rt-"+str(l)+"_"+s for s in portStri

### identify whether rat previously took the same or alternative path to the left available port, and if it was rewarded or not

In [5]:
for l in range(1,6):
    photrats.triframe.loc[:,"rt-"+str(l)+"_left"] = photrats.get_left_val(photrats.triframe,factor="rt-"+str(l))
photrats.triframe.loc[:,"samePath_t-1_left"] = photrats.get_left_val(photrats.triframe,factor="samePath_t-1")

### save as csv for regression analysis in R

In [6]:
photrats.triframe.to_csv(photrats.directory_prefix+"triframe4sameValtLeftChoiceReg.csv")

### create a dictionary of reward vs omission choice probabilities when rat previously took same vs alternative path

In [7]:
ratSameValtChoiceProbs,ratNs = create_sameValtChoiceDict(photrats)

### test if same and alt are sig dif from each other

In [8]:
sameVals = [np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmSame"]) for rat in ratSameValtChoiceProbs.keys()]
altVals = [np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmAlt"]) for rat in ratSameValtChoiceProbs.keys()]
wilcoxon(sameVals,altVals)

WilcoxonResult(statistic=16.0, pvalue=0.275390625)

### visualize result

In [9]:
from photometryQuantifications import plot_sigMarkers

In [10]:
fig = plotChoosePortSameVAlt(photrats,ratSameValtChoiceProbs)
plt.ylim(-.3,.3)
plt.yticks(np.arange(-3,4)/10)

55.0 0.0009765625
52.0 0.0048828125


C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1765: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  order = pd.unique(vector)


([<matplotlib.axis.YTick at 0x1f355351f10>,
 [Text(0, -0.3, '−0.3'),
  Text(0, -0.2, '−0.2'),
  Text(0, -0.1, '−0.1'),
  Text(0, 0.0, '0.0'),
  Text(0, 0.1, '0.1'),
  Text(0, 0.2, '0.2'),
  Text(0, 0.3, '0.3')])

# visualize individual rat sensitivities to costs and benefits

In [ ]:
photrats.get_log_pchoos_v_costNben()
photrats.regdf = photrats.regdf.loc[photrats.regdf.tri>25,]

In [ ]:
photrats.regdf

# get and save dataframe for mixed-effects p(choose L) regression in R

In [ ]:
photrats.regdf.to_csv(photrats.directory_prefix+"\\costVbenDf4reg.csv")

### load regression results from R

In [24]:
regCoefs = pd.read_csv(photrats.directory_prefix+"\\chooseLregCoefsByRat.csv")
regSum = pd.read_csv(photrats.directory_prefix+"\\chooseLregSummary.csv")

### plot regression results from R

In [19]:
regSum

,Unnamed: 0,Estimate,Std. Error,z value,Pr(>|z|)
0,(Intercept),2.222081,0.307533,7.225502,4.992534e-13
1,pRwdScaled,1.603194,0.162157,9.886660,4.756312e-23
2,ldifScaled,-6.807293,0.550069,-12.375344,3.553706e-35


In [21]:
regCoefs

,Unnamed: 0,(Intercept),pRwdScaled,ldifScaled
0,0,2.357369,1.552750,-6.989187
1,1,2.178022,1.591268,-6.727723
2,2,2.386142,1.514567,-7.008159
3,3,1.806065,1.527834,-6.082627
4,4,1.901926,1.770652,-6.411339
5,5,2.392983,1.763149,-7.197530
6,6,2.894191,1.435554,-7.770432
7,7,1.920301,1.757092,-6.431260
8,8,1.827173,1.626431,-6.187385
9,9,2.402928,1.426401,-6.971966


In [35]:
fig = plt.figure(figsize=(3.5,4))
xvals = ["p(reward)","Distance cost"]
sns.barplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='darkgrey',alpha=.5)
sns.stripplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='k',size=8,marker='X',alpha=.9, jitter=True)
plt.axhline(y=0,ls='--',color='k')
plt.xticks([0,1],xvals,fontweight="bold")
plotRegSigLevel(regSum,0,2.0,1)
plotRegSigLevel(regSum,1,2.0,2)
plt.ylim(-8,3)
plt.yticks(np.arange(-7.5, 5, 2.5))
plt.ylabel("Choice ß value",fontsize='xx-large',fontweight='bold')
plt.tight_layout()

C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\categorical.py:166: FutureWarning: Setting a gradient palette using color= is deprecated and will be removed in version 0.13. Set `palette='dark:k'` for same effect.
  warnings.warn(msg, FutureWarning)
C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
